## 과제 1

- 아래의 코드를 처음부터(import 구문 부터 끝까지 안보고 혼자서 작성할 수 있을 때까지 반복해보세요.)
- 주석 빼고 22줄입니다.
- 빈 코드에서 안보고 처음부터 끝까지 외워서 할 수 있다면 과제를 완수한 것입니다.
- 중간에 코드가 이해되지 않는다면 강의를 찾아서 그 부분만 다시보세요.
- 주석은 작성하지 않아도 좋습니다.


하루에 최소 5번은 쳐보세요. 그러면 눈에 익숙해집니다.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub

# 단계 1: 문서 로드(Load Documents)
loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("teddynote/rag-korean")

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
# 실행
chain.invoke("삼성전자가 개발한 생성형 AI 이름은?")

"삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."